In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# Hyperparameters
input_size = 784  # 28x28 images
hidden_size = 128
hidden_size2 = 64
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.001

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 38.1MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.19MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 9.22MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.86MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss, and optimizer
model = MLP(input_size, hidden_size, hidden_size2, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Epoch [1/5], Step [100/938], Loss: 0.3474
Epoch [1/5], Step [200/938], Loss: 0.2454
Epoch [1/5], Step [300/938], Loss: 0.3262
Epoch [1/5], Step [400/938], Loss: 0.3719
Epoch [1/5], Step [500/938], Loss: 0.3901
Epoch [1/5], Step [600/938], Loss: 0.4915
Epoch [1/5], Step [700/938], Loss: 0.4347
Epoch [1/5], Step [800/938], Loss: 0.4257
Epoch [1/5], Step [900/938], Loss: 0.3904
Epoch [2/5], Step [100/938], Loss: 0.1899
Epoch [2/5], Step [200/938], Loss: 0.2425
Epoch [2/5], Step [300/938], Loss: 0.0872
Epoch [2/5], Step [400/938], Loss: 0.2345
Epoch [2/5], Step [500/938], Loss: 0.2098
Epoch [2/5], Step [600/938], Loss: 0.1731
Epoch [2/5], Step [700/938], Loss: 0.2317
Epoch [2/5], Step [800/938], Loss: 0.2732
Epoch [2/5], Step [900/938], Loss: 0.3121
Epoch [3/5], Step [100/938], Loss: 0.0956
Epoch [3/5], Step [200/938], Loss: 0.1668
Epoch [3/5], Step [300/938], Loss: 0.2143
Epoch [3/5], Step [400/938], Loss: 0.2006
Epoch [3/5], Step [500/938], Loss: 0.1484
Epoch [3/5], Step [600/938], Loss: